In [1]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from glob import glob

main_path = "/disk1/luna16/"
working_path = "/disk1/luna16/output/"

file_list = glob(working_path + "lungmask_*.npy")
out_images = []  # final set of images
out_nodemasks = []  # final set of nodemasks
for fname in file_list:
    print("working on file ", fname)
    imgs_to_process = np.load(fname.replace("lungmask", "images"))
    masks = np.load(fname)
    node_masks = np.load(fname.replace("lungmask", "masks"))
    for i in range(len(imgs_to_process)):
        mask = masks[i]
        node_mask = node_masks[i]
        img = imgs_to_process[i]
        new_size = [512, 512]  # we're scaling back up to the original size of the image
        img = mask * img  # apply lung mask
        #
        # renormalizing the masked image (in the mask region)
        #
        new_mean = np.mean(img[mask > 0])
        new_std = np.std(img[mask > 0])
        #
        #  Pulling the background color up to the lower end
        #  of the pixel range for the lungs
        #
        old_min = np.min(img)  # background color
        img[img == old_min] = new_mean - 1.2 * new_std  # resetting backgound color
        img = img - new_mean
        img = img / new_std
        # make image bounding box  (min row, min col, max row, max col)
        labels = measure.label(mask)
        regions = measure.regionprops(labels)
        #
        # Finding the global min and max row over all regions
        #
        min_row = 512
        max_row = 0
        min_col = 512
        max_col = 0
        for prop in regions:
            B = prop.bbox
            if min_row > B[0]:
                min_row = B[0]
            if min_col > B[1]:
                min_col = B[1]
            if max_row < B[2]:
                max_row = B[2]
            if max_col < B[3]:
                max_col = B[3]
        width = max_col - min_col
        height = max_row - min_row
        if width > height:
            max_row = min_row + width
        else:
            max_col = min_col + height
        #
        # cropping the image down to the bounding box for all regions
        # (there's probably an skimage command that can do this in one line)
        #
        img = img[min_row:max_row, min_col:max_col]
        mask = mask[min_row:max_row, min_col:max_col]
        if max_row - min_row < 5 or max_col - min_col < 5:  # skipping all images with no god regions
            pass
        else:
            # moving range to -1 to 1 to accomodate the resize function
            mean = np.mean(img)
            img = img - mean
            min = np.min(img)
            max = np.max(img)
            img = img / (max - min)
            new_img = resize(img, [512, 512])
            new_node_mask = resize(node_mask[min_row:max_row, min_col:max_col], [512, 512])
            out_images.append(new_img)
            out_nodemasks.append(new_node_mask)

num_images = len(out_images)
#
#  Writing out images and masks as 1 channel arrays for input into network
#
final_images = np.ndarray([num_images, 1, 512, 512], dtype=np.float32)
final_masks = np.ndarray([num_images, 1, 512, 512], dtype=np.float32)
for i in range(num_images):
    final_images[i, 0] = out_images[i]
    final_masks[i, 0] = out_nodemasks[i]

rand_i = np.random.choice(range(num_images), size=num_images, replace=False)
test_i = int(0.1*num_images)

np.save(main_path+"trainImages.npy",final_images[rand_i[test_i:]])
np.save(main_path+"trainMasks.npy",final_masks[rand_i[test_i:]])

np.save(main_path+"testImages.npy",final_images[rand_i[:test_i]])
np.save(main_path+"testMasks.npy",final_masks[rand_i[:test_i]])
print('process done...')

working on file  /disk1/luna16/output/lungmask_0271_0942.npy


/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


working on file  /disk1/luna16/output/lungmask_0336_1158.npy
working on file  /disk1/luna16/output/lungmask_0718_0311.npy
working on file  /disk1/luna16/output/lungmask_0019_0434.npy
working on file  /disk1/luna16/output/lungmask_0586_0227.npy
working on file  /disk1/luna16/output/lungmask_0209_0938.npy
working on file  /disk1/luna16/output/lungmask_0089_0570.npy
working on file  /disk1/luna16/output/lungmask_0080_0958.npy
working on file  /disk1/luna16/output/lungmask_0726_0056.npy
working on file  /disk1/luna16/output/lungmask_0448_0552.npy
working on file  /disk1/luna16/output/lungmask_0649_0773.npy
working on file  /disk1/luna16/output/lungmask_0419_1070.npy
working on file  /disk1/luna16/output/lungmask_0063_0699.npy
working on file  /disk1/luna16/output/lungmask_0233_1045.npy
working on file  /disk1/luna16/output/lungmask_0094_0604.npy
working on file  /disk1/luna16/output/lungmask_0252_0717.npy
working on file  /disk1/luna16/output/lungmask_0441_0002.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0536_0668.npy
working on file  /disk1/luna16/output/lungmask_0794_0198.npy
working on file  /disk1/luna16/output/lungmask_0427_0735.npy
working on file  /disk1/luna16/output/lungmask_0129_0104.npy
working on file  /disk1/luna16/output/lungmask_0403_0173.npy
working on file  /disk1/luna16/output/lungmask_0461_0406.npy
working on file  /disk1/luna16/output/lungmask_0028_0654.npy
working on file  /disk1/luna16/output/lungmask_0453_0079.npy
working on file  /disk1/luna16/output/lungmask_0666_0358.npy
working on file  /disk1/luna16/output/lungmask_0266_0305.npy
working on file  /disk1/luna16/output/lungmask_0051_1075.npy
working on file  /disk1/luna16/output/lungmask_0315_0732.npy
working on file  /disk1/luna16/output/lungmask_0690_0159.npy
working on file  /disk1/luna16/output/lungmask_0215_0281.npy
working on file  /disk1/luna16/output/lungmask_0539_0936.npy
working on file  /disk1/luna16/output/lungmask_0034_0086.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0674_1094.npy
working on file  /disk1/luna16/output/lungmask_0655_0739.npy
working on file  /disk1/luna16/output/lungmask_0182_0551.npy
working on file  /disk1/luna16/output/lungmask_0351_0708.npy
working on file  /disk1/luna16/output/lungmask_0725_0545.npy
working on file  /disk1/luna16/output/lungmask_0141_0424.npy
working on file  /disk1/luna16/output/lungmask_0619_0769.npy
working on file  /disk1/luna16/output/lungmask_0450_0647.npy
working on file  /disk1/luna16/output/lungmask_0415_1059.npy
working on file  /disk1/luna16/output/lungmask_0449_0368.npy
working on file  /disk1/luna16/output/lungmask_0183_0327.npy
working on file  /disk1/luna16/output/lungmask_0102_0596.npy
working on file  /disk1/luna16/output/lungmask_0744_0798.npy
working on file  /disk1/luna16/output/lungmask_0363_0120.npy
working on file  /disk1/luna16/output/lungmask_0599_0416.npy
working on file  /disk1/luna16/output/lungmask_0677_0345.npy
working on file  /disk1/

...

working on file  /disk1/luna16/output/lungmask_0180_0762.npy
working on file  /disk1/luna16/output/lungmask_0094_0603.npy
working on file  /disk1/luna16/output/lungmask_0130_0099.npy
working on file  /disk1/luna16/output/lungmask_0163_0776.npy
working on file  /disk1/luna16/output/lungmask_0686_0396.npy
working on file  /disk1/luna16/output/lungmask_0101_0389.npy
working on file  /disk1/luna16/output/lungmask_0300_0684.npy
working on file  /disk1/luna16/output/lungmask_0180_0765.npy
working on file  /disk1/luna16/output/lungmask_0328_0638.npy
working on file  /disk1/luna16/output/lungmask_0787_0828.npy
working on file  /disk1/luna16/output/lungmask_0203_0112.npy
working on file  /disk1/luna16/output/lungmask_0639_1184.npy
working on file  /disk1/luna16/output/lungmask_0092_0330.npy
working on file  /disk1/luna16/output/lungmask_0497_0257.npy
working on file  /disk1/luna16/output/lungmask_0131_0755.npy
working on file  /disk1/luna16/output/lungmask_0288_0934.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0727_0381.npy
working on file  /disk1/luna16/output/lungmask_0791_0022.npy
working on file  /disk1/luna16/output/lungmask_0461_0407.npy
working on file  /disk1/luna16/output/lungmask_0085_0886.npy
working on file  /disk1/luna16/output/lungmask_0567_0997.npy
working on file  /disk1/luna16/output/lungmask_0469_0367.npy
working on file  /disk1/luna16/output/lungmask_0220_1125.npy
working on file  /disk1/luna16/output/lungmask_0468_0873.npy
working on file  /disk1/luna16/output/lungmask_0791_0021.npy
working on file  /disk1/luna16/output/lungmask_0290_0912.npy
working on file  /disk1/luna16/output/lungmask_0065_0249.npy
working on file  /disk1/luna16/output/lungmask_0759_0920.npy
working on file  /disk1/luna16/output/lungmask_0175_1091.npy
working on file  /disk1/luna16/output/lungmask_0141_0427.npy
working on file  /disk1/luna16/output/lungmask_0516_1076.npy
working on file  /disk1/luna16/output/lungmask_0127_0046.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0040_1033.npy
working on file  /disk1/luna16/output/lungmask_0238_1087.npy
working on file  /disk1/luna16/output/lungmask_0285_0300.npy
working on file  /disk1/luna16/output/lungmask_0400_0395.npy
working on file  /disk1/luna16/output/lungmask_0690_0161.npy
working on file  /disk1/luna16/output/lungmask_0445_0342.npy
working on file  /disk1/luna16/output/lungmask_0052_0824.npy
working on file  /disk1/luna16/output/lungmask_0148_0096.npy
working on file  /disk1/luna16/output/lungmask_0455_0090.npy
working on file  /disk1/luna16/output/lungmask_0440_0236.npy
working on file  /disk1/luna16/output/lungmask_0489_0024.npy
working on file  /disk1/luna16/output/lungmask_0000_0531.npy
working on file  /disk1/luna16/output/lungmask_0370_1089.npy
working on file  /disk1/luna16/output/lungmask_0324_0148.npy
working on file  /disk1/luna16/output/lungmask_0327_0598.npy
working on file  /disk1/luna16/output/lungmask_0272_0860.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0050_0565.npy
working on file  /disk1/luna16/output/lungmask_0699_0400.npy
working on file  /disk1/luna16/output/lungmask_0233_1044.npy
working on file  /disk1/luna16/output/lungmask_0211_1008.npy
working on file  /disk1/luna16/output/lungmask_0188_0628.npy
working on file  /disk1/luna16/output/lungmask_0720_0388.npy
working on file  /disk1/luna16/output/lungmask_0649_0771.npy
working on file  /disk1/luna16/output/lungmask_0127_0044.npy
working on file  /disk1/luna16/output/lungmask_0205_0357.npy
working on file  /disk1/luna16/output/lungmask_0611_0479.npy
working on file  /disk1/luna16/output/lungmask_0282_0992.npy
working on file  /disk1/luna16/output/lungmask_0297_1149.npy
working on file  /disk1/luna16/output/lungmask_0292_1035.npy
working on file  /disk1/luna16/output/lungmask_0647_0812.npy
working on file  /disk1/luna16/output/lungmask_0084_1097.npy
working on file  /disk1/luna16/output/lungmask_0383_0999.npy
working on file  /disk1/

working on file  /disk1/luna16/output/lungmask_0079_0909.npy
working on file  /disk1/luna16/output/lungmask_0133_1043.npy
working on file  /disk1/luna16/output/lungmask_0649_0770.npy
working on file  /disk1/luna16/output/lungmask_0751_0218.npy
working on file  /disk1/luna16/output/lungmask_0691_0200.npy
working on file  /disk1/luna16/output/lungmask_0476_0312.npy
working on file  /disk1/luna16/output/lungmask_0599_0414.npy
working on file  /disk1/luna16/output/lungmask_0067_1181.npy
working on file  /disk1/luna16/output/lungmask_0570_0737.npy
working on file  /disk1/luna16/output/lungmask_0782_0282.npy
working on file  /disk1/luna16/output/lungmask_0545_0602.npy
working on file  /disk1/luna16/output/lungmask_0314_0371.npy
working on file  /disk1/luna16/output/lungmask_0420_0959.npy
working on file  /disk1/luna16/output/lungmask_0740_0651.npy
working on file  /disk1/luna16/output/lungmask_0758_0117.npy
working on file  /disk1/luna16/output/lungmask_0235_0316.npy
process done...


In [5]:
import numpy as np

main_path = "/disk1/luna16/"
working_path = "/disk1/luna16/output/"

imgs_train = np.load(main_path + "trainImages.npy").astype(np.float32)
imgs_mask_train = np.load(main_path + "trainMasks.npy").astype(np.float32)

In [7]:
imgs_train.shape

(2850, 1, 512, 512)